# 3. Run second-level RSA on the cross-validated Feedback History RSA clusters

## Goal

We first found the two important cortical ROIs encoding feedback histories, the MiOG and IFG clusters. We thought that these clusters might represent other task-relevant information. Therefore, we conducted second-level RSA on these clusters to identify whether the feedback history clusters could encode other task-related aspects, especially the spatial information.

## Details

In [1]:
# Imports

from pathlib import Path
from typing import Literal
import os

import numpy as np
import pandas as pd

from nilearn.image import load_img

from rsatoolbox.rdm import compare_rho_a
from scipy.spatial.distance import pdist
from scipy.stats import (
    permutation_test,
    ttest_1samp,
)
from sklearn.linear_model import LinearRegression

In [2]:
# Paths

# Cluster mask directory path
GIT_ROOT_LINES = !git rev-parse --show-toplevel
GIT_ROOT = Path(GIT_ROOT_LINES[0])
feedback_history_cluster_mask_dir = (
    GIT_ROOT
    / "second-level"
    / "output"
    / "cross_validated_cluster_mask"
    / "recent_2_and_recent_3_trial"
)

# Feedback history RSA cluster masks
feedback_history_cluster_mask_list = [
    (
        "1_MiOG",
        load_img(
            feedback_history_cluster_mask_dir
            / "feedback_history_cluster_1_MiOG_mask.nii"
        )
        .get_fdata()
        .astype(bool),
    ),
    (
        "2_IFG",
        load_img(
            feedback_history_cluster_mask_dir
            / "feedback_history_cluster_2_IFG_mask.nii"
        )
        .get_fdata()
        .astype(bool),
    ),
]

# Data paths
discovery_first_level_output_path = Path(
    "/users/jss/data/ISL_photographer/2022/bids/derivatives/first-level/"  # Please update accordingly
)
validation_first_level_output_path = Path(
    "/users/jss/data/ISL_photographer/2023/bids/derivatives/first-level"  # Please update accordingly
)
subgroup_first_level_output_path_dict = {
    "discovery": discovery_first_level_output_path,
    "validation": validation_first_level_output_path,
}

# Result dataframe output path
rsa_result_output_path = GIT_ROOT / "second-level" / "output" / "second_level_rsa"

# Subject ID lists
discovery_subject_list = [
    "sub-09",
    "sub-10",
    "sub-16",
    "sub-17",
    "sub-19",
    "sub-20",
    "sub-21",
    "sub-22",
    "sub-23",
    "sub-24",
    "sub-26",
    "sub-27",
    "sub-28",
    "sub-29",
    "sub-30",
    "sub-31",
]

validation_subject_list = [
    "sub-32",
    "sub-33",
    "sub-34",
    "sub-35",
    "sub-36",
    "sub-37",
    "sub-38",
    "sub-39",
    "sub-41",
    "sub-42",
    "sub-45",
    "sub-47",
    "sub-48",
    "sub-49",
    "sub-50",
    "sub-51",
]

subgroup_subject_list_dict = {
    "discovery": discovery_subject_list,
    "validation": validation_subject_list,
}

In [3]:
# Behavioral data
# Preprocessed exploration-related behavioral data
# Note: For subgroup IDs, we used "2022" for the Discovery group and "2023" for the Validation group
# `exploration_time`: Elapsed time from the exploration start to capture
# `exploration_distance`: Distance between the start and captue points
# `capture_action`: Whether the participant captured a scene within 40 s time limit ('capture') or
#                   failed to capture within the time limit ('capture_failed')
# `capture_lat` / `capture_lon`: Latitude/Longitude for each capture location

exploration_info_df = pd.read_csv(
    GIT_ROOT / "second-level" / "data" / "photographer_exploration_info.csv"
)
exploration_info_df.head()

,subgroup,subject_id,run_id,city,trial,exploration_time,exploration_distance,capture_action,capture_lat,capture_lon
0,2022,sub-09,run-01,Boston,1,1.710,0.0048,capture,42.359721,-71.059837
1,2022,sub-09,run-01,Boston,2,27.062,0.1169,capture,42.360616,-71.060584
2,2022,sub-09,run-01,Boston,3,30.895,0.1266,capture,42.359631,-71.059813
3,2022,sub-09,run-01,Boston,4,15.715,0.0477,capture,42.359269,-71.059501
4,2022,sub-09,run-01,Boston,5,16.853,0.0103,capture,42.359315,-71.059610


In [4]:
all_model_rdm_list = [
    # feedback history model names
    "current_trial",
    "one_back_trial",
    "two_back_trial",
    "recent_2_trial",
    "recent_3_trial",
    "previous_2_trial",
    # exploration model names
    "exploration_time",
    "exploration_distance",
    "capture_distance",
]


# Haversine formula to compute a distance between two coordinates on the earth
# REF: https://www.movable-type.co.uk/scripts/latlong.html
def haversine(coord_1: tuple[float, float], coord_2: tuple[float, float]):
    R = 6371.0088

    lat_1 = np.radians(coord_1[0])
    lon_1 = np.radians(coord_1[1])

    lat_2 = np.radians(coord_2[0])
    lon_2 = np.radians(coord_2[1])

    d_lat = lat_2 - lat_1
    d_lon = lon_2 - lon_1

    a = (np.sin(d_lat / 2) * np.sin(d_lat / 2)) + np.cos(lat_1) * np.cos(lat_2) * (
        np.sin(d_lon / 2) * np.sin(d_lon / 2)
    )
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    distance = R * c
    return distance


# A helper function to load feedback history/exploration model RDMs
def load_model_rdm_vector(
    rdm_name: str,
    subgroup: Literal["discovery", "validation"],
    subject_id: str,
    run_id: str,
    exploration_info_df=exploration_info_df,
):
    if rdm_name not in all_model_rdm_list:
        raise ValueError

    if rdm_name in all_model_rdm_list[:6]:  # Feedback history model RDMs
        subgroup_first_level_output_path = subgroup_first_level_output_path_dict[
            subgroup
        ]

        subject_model_rdm_dir = (
            subgroup_first_level_output_path
            / subject_id
            / "rsa_model_rdm"
            / "feedback_model"
        )

        model_rdm_path = (
            subject_model_rdm_dir
            / f"{subject_id}_{run_id}_task-photographer_{rdm_name}_vector.npy"
        )
        return np.load(model_rdm_path)

    elif rdm_name in all_model_rdm_list[6:]:  # Exploration model RDMs
        run_exploration_df = exploration_info_df[
            (exploration_info_df["subject_id"] == subject_id)
            & (exploration_info_df["run_id"] == run_id)
        ]

        if (rdm_name == "exploration_time") or (rdm_name == "exploration_distance"):
            run_exploration_data_list = run_exploration_df[rdm_name].tolist()
            run_exploration_data_vector = [
                [t] for t in run_exploration_data_list[2:]
            ]  # Using Trial 3 to 8
            return pdist(run_exploration_data_vector)  # Euclidean distance

        elif rdm_name == "capture_distance":
            run_exploration_coord_vector = [
                (lat, lon)
                for lat, lon in zip(
                    run_exploration_df["capture_lat"][2:],
                    run_exploration_df["capture_lon"][2:],
                )
            ]
            return pdist(run_exploration_coord_vector, metric=haversine)

### Second-level RSA using exploration models

In [5]:
# A helper function to compute neural RDM vector
def compute_neural_rdm_vector(neural_beta_array: np.ndarray):
    neural_rdm_vector = pdist(neural_beta_array, "correlation")
    neural_rdm_vector = np.nan_to_num(
        neural_rdm_vector, nan=1.0
    )  # correlation distances can be nan!

    return neural_rdm_vector


# Main function to create second-level RSA dataframe from a cluster mask and subgroup name
def create_exploration_second_level_rsa_df(
    cluster_mask: tuple[str, np.ndarray], subgroup: Literal["discovery", "validation"]
):
    cluster_name, cluster_mask = cluster_mask

    print(f"Processing {cluster_name} (Subgroup: {subgroup})...")

    exploration_model_list = all_model_rdm_list[6:]
    run_id_list = ["run-01", "run-02", "run-03", "run-04", "run-05"]
    trial_id_list = [
        "trial3",
        "trial4",
        "trial5",
        "trial6",
        "trial7",
        "trial8",
    ]  # Using Trial 3 to Trial 8

    subject_list = subgroup_subject_list_dict[subgroup]
    subgroup_first_level_output_dir = subgroup_first_level_output_path_dict[subgroup]

    rsa_result_dict_list: list[dict[str, list]] = []

    for subject_id in subject_list:
        subject_feedback_beta_dir = (
            subgroup_first_level_output_dir
            / subject_id
            / "rsa_neural_data"
            / "feedback_beta"
        )

        subject_rsa_dict: dict[str, list] = {}

        for model_rdm in exploration_model_list:
            subject_rsa_dict[model_rdm] = []

        for run_id in run_id_list:
            # 1. Compute neural rdm vector
            subject_run_feedback_beta_list = []

            for trial_id in trial_id_list:
                norm_beta_path = (
                    subject_feedback_beta_dir
                    / f"{subject_id}_task-photographer_{run_id}_{trial_id}_feedback_beta.nii"
                )
                norm_beta_array = load_img(norm_beta_path).get_fdata()
                masked_beta_array = norm_beta_array[cluster_mask]
                subject_run_feedback_beta_list.append(masked_beta_array)

            subject_run_feedback_beta_array = np.vstack(subject_run_feedback_beta_list)
            subject_run_feedback_neural_rdm_vector = compute_neural_rdm_vector(
                subject_run_feedback_beta_array
            )

            # 2. Load model rdm vector and perform RSA
            for model_rdm in exploration_model_list:
                subject_run_model_rdm_vector = load_model_rdm_vector(
                    model_rdm, subgroup, subject_id, run_id
                )

                subject_run_rsa_corr = compare_rho_a(
                    subject_run_feedback_neural_rdm_vector, subject_run_model_rdm_vector
                )[0][0]  # Spearman's rank correlation
                subject_run_rsa_corr = np.arctanh(
                    subject_run_rsa_corr
                )  # Fisher's r-to-z transform
                subject_rsa_dict[model_rdm].append(subject_run_rsa_corr)

        subject_rsa_result_dict = {
            "subgroup": subgroup,
            "subject_id": subject_id,
            "cluster": cluster_name,
        }

        # Average individual RSA correlations across runs
        for model_rdm in exploration_model_list:
            subject_run_rsa_corr_list = subject_rsa_dict[model_rdm]
            assert len(subject_run_rsa_corr_list) == len(run_id_list)
            subject_mean_rsa_corr = np.mean(subject_run_rsa_corr_list)

            subject_rsa_result_dict[model_rdm] = subject_mean_rsa_corr

        rsa_result_dict_list.append(subject_rsa_result_dict)

    second_level_RSA_df = pd.DataFrame.from_records(rsa_result_dict_list)

    return second_level_RSA_df

In [6]:
# Helper functions to perform permutation-based one-sample t-tests on a numpy array
def t_statistic(x: np.ndarray, axis=1):
    return ttest_1samp(x, popmean=0, axis=axis).statistic


def permute_1samp(x: np.ndarray, alternative: Literal["two-sided", "less", "greater"]):
    rng = np.random.RandomState(605)
    return permutation_test(
        (x,),
        t_statistic,
        permutation_type="samples",
        n_resamples=10000,
        random_state=rng,
        alternative=alternative,
    )


# Performing permuation tests (one-sample t-tests, one-sided) for a given second-level RSA results df and model RDM candidates
def perm_test_on_second_level_rsa(rsa_df: pd.DataFrame, model_rdm_list: list[str]):
    for rdm_model in model_rdm_list:
        print(f"{rdm_model} RDM")

        print(f"\tmean (z-scored) r = {round(np.mean(rsa_df[rdm_model]), 3)}")

        res = permute_1samp(rsa_df[rdm_model], alternative="greater")  # one-sided test
        print(
            f"\tt = {round(res.statistic, 3)} (P = {round(res.pvalue, 4)}) {'*' if res.pvalue < 0.05 else ''}"
        )

        d = np.mean(rsa_df[rdm_model]) / np.std(
            rsa_df[rdm_model]
        )  # standardized effect size
        print(f"\td = {round(d, 3)}")

        print()

In [7]:
# (1) MiOG cluster, Discovery group
exploration_second_level_MiOG_disovery_df = create_exploration_second_level_rsa_df(
    feedback_history_cluster_mask_list[0], "discovery"
)

exploration_second_level_rsa_output_dir = (
    rsa_result_output_path / "exploration_second_level"
)
os.makedirs(exploration_second_level_rsa_output_dir, exist_ok=True)

exploration_second_level_MiOG_disovery_df.to_csv(
    exploration_second_level_rsa_output_dir / "discovery_MiOG_second_level.csv",
    index=False,
)

print()
perm_test_on_second_level_rsa(
    exploration_second_level_MiOG_disovery_df, all_model_rdm_list[6:]
)

Processing 1_MiOG (Subgroup: discovery)...

exploration_time RDM
	mean (z-scored) r = -0.043
	t = -1.364 (P = 0.9044) 
	d = -0.352

exploration_distance RDM
	mean (z-scored) r = -0.049
	t = -1.557 (P = 0.9286) 
	d = -0.402

capture_distance RDM
	mean (z-scored) r = 0.282
	t = 9.45 (P = 0.0001) *
	d = 2.44



In [8]:
# (2) MiOG cluster, Validation group
exploration_second_level_MiOG_validation_df = create_exploration_second_level_rsa_df(
    feedback_history_cluster_mask_list[0], "validation"
)

exploration_second_level_MiOG_validation_df.to_csv(
    exploration_second_level_rsa_output_dir / "validation_MiOG_second_level.csv",
    index=False,
)

print()
perm_test_on_second_level_rsa(
    exploration_second_level_MiOG_validation_df, all_model_rdm_list[6:]
)

Processing 1_MiOG (Subgroup: validation)...

exploration_time RDM
	mean (z-scored) r = 0.002
	t = 0.058 (P = 0.4806) 
	d = 0.015

exploration_distance RDM
	mean (z-scored) r = 0.02
	t = 0.649 (P = 0.2602) 
	d = 0.168

capture_distance RDM
	mean (z-scored) r = 0.309
	t = 6.917 (P = 0.0001) *
	d = 1.786



In [9]:
# (3) IFG cluster, Discovery group
exploration_second_level_IFG_disovery_df = create_exploration_second_level_rsa_df(
    feedback_history_cluster_mask_list[1], "discovery"
)

exploration_second_level_IFG_disovery_df.to_csv(
    exploration_second_level_rsa_output_dir / "discovery_IFG_second_level.csv",
    index=False,
)

print()
perm_test_on_second_level_rsa(
    exploration_second_level_IFG_disovery_df, all_model_rdm_list[6:]
)

Processing 2_IFG (Subgroup: discovery)...

exploration_time RDM
	mean (z-scored) r = -0.068
	t = -2.295 (P = 0.98) 
	d = -0.593

exploration_distance RDM
	mean (z-scored) r = -0.044
	t = -1.378 (P = 0.9049) 
	d = -0.356

capture_distance RDM
	mean (z-scored) r = 0.209
	t = 5.981 (P = 0.0003) *
	d = 1.544



In [10]:
# (4) IFG cluster, Validation group
exploration_second_level_IFG_validation_df = create_exploration_second_level_rsa_df(
    feedback_history_cluster_mask_list[1], "validation"
)

exploration_second_level_IFG_validation_df.to_csv(
    exploration_second_level_rsa_output_dir / "validation_IFG_second_level.csv",
    index=False,
)

print()
perm_test_on_second_level_rsa(
    exploration_second_level_IFG_validation_df, all_model_rdm_list[6:]
)

Processing 2_IFG (Subgroup: validation)...

exploration_time RDM
	mean (z-scored) r = 0.009
	t = 0.324 (P = 0.3819) 
	d = 0.084

exploration_distance RDM
	mean (z-scored) r = 0.037
	t = 1.481 (P = 0.0809) 
	d = 0.382

capture_distance RDM
	mean (z-scored) r = 0.255
	t = 6.806 (P = 0.0001) *
	d = 1.757



### Partial correlation-based second-level RSA on feedback history models (controlling for the Capture Distance model)

In [11]:
# Main function to create partial correlation-based second-level RSA dataframe
# We regress out the Capture Distance model from both neural and feedback history model RDMs
def create_partial_correlation_feedback_history_second_level_rsa_df(
    cluster_mask: tuple[str, np.ndarray], subgroup: Literal["discovery", "validation"]
):
    cluster_name, cluster_mask = cluster_mask

    print(f"Processing {cluster_name} (Subgroup: {subgroup})...")

    feedback_history_model_rdm_list = all_model_rdm_list[:6]
    run_id_list = ["run-01", "run-02", "run-03", "run-04", "run-05"]
    trial_id_list = ["trial3", "trial4", "trial5", "trial6", "trial7", "trial8"]

    subject_list = subgroup_subject_list_dict[subgroup]
    subgroup_first_level_output_dir = subgroup_first_level_output_path_dict[subgroup]

    rsa_result_dict_list: list[dict[str, list]] = []

    for subject_id in subject_list:
        subject_feedback_beta_dir = (
            subgroup_first_level_output_dir
            / subject_id
            / "rsa_neural_data"
            / "feedback_beta"
        )

        subject_rsa_dict: dict[str, list] = {}

        for rdm_model in feedback_history_model_rdm_list:
            subject_rsa_dict[rdm_model] = []

        for run_id in run_id_list:
            # 1. Compute neural rdm vector
            subject_run_feedback_beta_list = []

            for trial_id in trial_id_list:
                norm_beta_path = (
                    subject_feedback_beta_dir
                    / f"{subject_id}_task-photographer_{run_id}_{trial_id}_feedback_beta.nii"
                )
                norm_beta_array = load_img(norm_beta_path).get_fdata()
                masked_beta_array = norm_beta_array[cluster_mask]
                subject_run_feedback_beta_list.append(masked_beta_array)

            subject_run_feedback_beta_array = np.vstack(subject_run_feedback_beta_list)
            subject_run_feedback_neural_rdm_vector = compute_neural_rdm_vector(
                subject_run_feedback_beta_array
            )

            # 2. Load the Capture Distance rdm vector
            subject_run_capture_distance_vector = load_model_rdm_vector(
                "capture_distance", subgroup, subject_id, run_id
            )

            # 3. load model rdm vector and perform partial correlation RSA
            for rdm_model in feedback_history_model_rdm_list:
                subject_run_model_rdm_vector = load_model_rdm_vector(
                    rdm_model, subgroup, subject_id, run_id
                )

                fit_neural = LinearRegression(fit_intercept=True)
                fit_model = LinearRegression(fit_intercept=True)

                fit_neural.fit(
                    subject_run_capture_distance_vector.reshape(-1, 1),
                    subject_run_feedback_neural_rdm_vector,
                )
                fit_model.fit(
                    subject_run_capture_distance_vector.reshape(-1, 1),
                    subject_run_model_rdm_vector,
                )

                # Regress out information associated with the Capture Distance model
                # from both model and neural RDM vectors
                residual_neural_vector = (
                    subject_run_feedback_neural_rdm_vector
                    - fit_neural.predict(
                        subject_run_capture_distance_vector.reshape(-1, 1)
                    )
                )
                residual_model_vector = (
                    subject_run_model_rdm_vector
                    - fit_model.predict(
                        subject_run_capture_distance_vector.reshape(-1, 1)
                    )
                )

                # Perform RSA between residual vectors
                subject_run_rsa_corr = compare_rho_a(
                    residual_neural_vector, residual_model_vector
                )[0][0]
                subject_run_rsa_corr = np.arctanh(subject_run_rsa_corr)
                subject_rsa_dict[rdm_model].append(subject_run_rsa_corr)

        subject_rsa_result_dict = {
            "subgroup": subgroup,
            "subject_id": subject_id,
            "cluster": cluster_name,
        }

        for rdm_model in feedback_history_model_rdm_list:
            subject_run_rsa_corr_list = subject_rsa_dict[rdm_model]
            assert len(subject_run_rsa_corr_list) == len(run_id_list)
            subject_mean_rsa_corr = np.mean(subject_run_rsa_corr_list)

            subject_rsa_result_dict[rdm_model] = subject_mean_rsa_corr

        rsa_result_dict_list.append(subject_rsa_result_dict)

    second_level_RSA_df = pd.DataFrame.from_records(rsa_result_dict_list)

    return second_level_RSA_df

In [12]:
# (1) MiOG cluster, Discovery group
partial_feedback_history_second_level_MiOG_disovery_df = (
    create_partial_correlation_feedback_history_second_level_rsa_df(
        feedback_history_cluster_mask_list[0], "discovery"
    )
)

partial_feedback_history_second_level_rsa_output_dir = (
    rsa_result_output_path / "partial_feedback_history_second_level"
)
os.makedirs(partial_feedback_history_second_level_rsa_output_dir, exist_ok=True)

partial_feedback_history_second_level_MiOG_disovery_df.to_csv(
    partial_feedback_history_second_level_rsa_output_dir
    / "discovery_MiOG_partial_second_level.csv",
    index=False,
)

print()
perm_test_on_second_level_rsa(
    partial_feedback_history_second_level_MiOG_disovery_df, all_model_rdm_list[:6]
)

Processing 1_MiOG (Subgroup: discovery)...

current_trial RDM
	mean (z-scored) r = 0.027
	t = 0.751 (P = 0.2393) 
	d = 0.194

one_back_trial RDM
	mean (z-scored) r = 0.032
	t = 1.468 (P = 0.0779) 
	d = 0.379

two_back_trial RDM
	mean (z-scored) r = -0.003
	t = -0.073 (P = 0.5324) 
	d = -0.019

recent_2_trial RDM
	mean (z-scored) r = 0.067
	t = 2.407 (P = 0.0171) *
	d = 0.621

recent_3_trial RDM
	mean (z-scored) r = 0.048
	t = 1.263 (P = 0.1177) 
	d = 0.326

previous_2_trial RDM
	mean (z-scored) r = 0.042
	t = 1.157 (P = 0.1291) 
	d = 0.299



In [13]:
# (2) MiOG cluster, Validation group
partial_feedback_history_second_level_MiOG_validation_df = (
    create_partial_correlation_feedback_history_second_level_rsa_df(
        feedback_history_cluster_mask_list[0], "validation"
    )
)

partial_feedback_history_second_level_MiOG_validation_df.to_csv(
    partial_feedback_history_second_level_rsa_output_dir
    / "validation_MiOG_partial_second_level.csv",
    index=False,
)

print()
perm_test_on_second_level_rsa(
    partial_feedback_history_second_level_MiOG_validation_df, all_model_rdm_list[:6]
)

Processing 1_MiOG (Subgroup: validation)...

current_trial RDM
	mean (z-scored) r = 0.038
	t = 1.225 (P = 0.1151) 
	d = 0.316

one_back_trial RDM
	mean (z-scored) r = 0.082
	t = 2.008 (P = 0.0339) *
	d = 0.518

two_back_trial RDM
	mean (z-scored) r = -0.014
	t = -0.452 (P = 0.6667) 
	d = -0.117

recent_2_trial RDM
	mean (z-scored) r = 0.115
	t = 3.354 (P = 0.0023) *
	d = 0.866

recent_3_trial RDM
	mean (z-scored) r = 0.107
	t = 3.111 (P = 0.0044) *
	d = 0.803

previous_2_trial RDM
	mean (z-scored) r = 0.078
	t = 2.142 (P = 0.0219) *
	d = 0.553



In [14]:
# (3) IFG cluster, Discovery group
partial_feedback_history_second_level_IFG_disovery_df = (
    create_partial_correlation_feedback_history_second_level_rsa_df(
        feedback_history_cluster_mask_list[1], "discovery"
    )
)

partial_feedback_history_second_level_IFG_disovery_df.to_csv(
    partial_feedback_history_second_level_rsa_output_dir
    / "discovery_IFG_partial_second_level.csv",
    index=False,
)

print()
perm_test_on_second_level_rsa(
    partial_feedback_history_second_level_IFG_disovery_df, all_model_rdm_list[:6]
)

Processing 2_IFG (Subgroup: discovery)...

current_trial RDM
	mean (z-scored) r = 0.042
	t = 1.431 (P = 0.0881) 
	d = 0.369

one_back_trial RDM
	mean (z-scored) r = 0.021
	t = 1.008 (P = 0.1849) 
	d = 0.26

two_back_trial RDM
	mean (z-scored) r = 0.019
	t = 0.571 (P = 0.2941) 
	d = 0.147

recent_2_trial RDM
	mean (z-scored) r = 0.064
	t = 2.702 (P = 0.0095) *
	d = 0.698

recent_3_trial RDM
	mean (z-scored) r = 0.073
	t = 3.211 (P = 0.0031) *
	d = 0.829

previous_2_trial RDM
	mean (z-scored) r = 0.041
	t = 1.252 (P = 0.1147) 
	d = 0.323



In [15]:
# (4) IFG cluster, Validation group
partial_feedback_history_second_level_IFG_validation_df = (
    create_partial_correlation_feedback_history_second_level_rsa_df(
        feedback_history_cluster_mask_list[1], "validation"
    )
)

partial_feedback_history_second_level_IFG_validation_df.to_csv(
    partial_feedback_history_second_level_rsa_output_dir
    / "validation_IFG_partial_second_level.csv",
    index=False,
)

print()
perm_test_on_second_level_rsa(
    partial_feedback_history_second_level_IFG_validation_df, all_model_rdm_list[:6]
)

Processing 2_IFG (Subgroup: validation)...

current_trial RDM
	mean (z-scored) r = 0.026
	t = 1.256 (P = 0.1118) 
	d = 0.324

one_back_trial RDM
	mean (z-scored) r = 0.102
	t = 2.388 (P = 0.018) *
	d = 0.617

two_back_trial RDM
	mean (z-scored) r = 0.018
	t = 0.439 (P = 0.33) 
	d = 0.113

recent_2_trial RDM
	mean (z-scored) r = 0.115
	t = 4.315 (P = 0.0004) *
	d = 1.114

recent_3_trial RDM
	mean (z-scored) r = 0.131
	t = 3.798 (P = 0.0014) *
	d = 0.981

previous_2_trial RDM
	mean (z-scored) r = 0.128
	t = 3.672 (P = 0.0003) *
	d = 0.948



### Partial correlation-based second-level RSA on Capture Distance model (controlling for either Recent-2 or Recent-3 Trial model)

In [17]:
# Main function to create partial correlation-based second-level RSA dataframe
# We regress out either Recent-2 or Recent-3 Trial model (feedback history models) from both neural and Capture Distance model RDMs
def create_partial_correlation_capture_distance_second_level_rsa_df(
    cluster_mask: tuple[str, np.ndarray], subgroup: Literal["discovery", "validation"]
):
    cluster_name, cluster_mask = cluster_mask

    print(f"Processing {cluster_name} (Subgroup {subgroup})...")

    feedback_history_model_rdm_list = [
        "recent_2_trial",
        "recent_3_trial",
    ]

    run_id_list = ["run-01", "run-02", "run-03", "run-04", "run-05"]
    trial_id_list = ["trial3", "trial4", "trial5", "trial6", "trial7", "trial8"]

    subject_list = subgroup_subject_list_dict[subgroup]
    subgroup_first_level_output_dir = subgroup_first_level_output_path_dict[subgroup]

    rsa_result_dict_list: list[dict[str, list]] = []

    for subject_id in subject_list:
        subject_feedback_beta_dir = (
            subgroup_first_level_output_dir
            / subject_id
            / "rsa_neural_data"
            / "feedback_beta"
        )

        subject_rsa_dict: dict[str, list] = {}

        for rdm_model in feedback_history_model_rdm_list:
            subject_rsa_dict[rdm_model] = []

        for run_id in run_id_list:
            # 1. Compute neural rdm vector
            subject_run_feedback_beta_list = []

            for trial_id in trial_id_list:
                norm_beta_path = (
                    subject_feedback_beta_dir
                    / f"{subject_id}_task-photographer_{run_id}_{trial_id}_feedback_beta.nii"
                )
                norm_beta_array = load_img(norm_beta_path).get_fdata()
                masked_beta_array = norm_beta_array[cluster_mask]
                subject_run_feedback_beta_list.append(masked_beta_array)

            subject_run_feedback_beta_array = np.vstack(subject_run_feedback_beta_list)
            subject_run_feedback_neural_rdm_vector = compute_neural_rdm_vector(
                subject_run_feedback_beta_array
            )

            # 2. Load the Capture Distance rdm vector
            subject_run_capture_distance_vector = load_model_rdm_vector(
                "capture_distance", subgroup, subject_id, run_id
            )

            # 3. Load model feedback history rdm vector and perform RSA
            for rdm_model in feedback_history_model_rdm_list:
                subject_run_model_rdm_vector = load_model_rdm_vector(
                    rdm_model, subgroup, subject_id, run_id
                )

                fit_neural = LinearRegression(fit_intercept=True)
                fit_capture = LinearRegression(fit_intercept=True)

                fit_neural.fit(
                    subject_run_model_rdm_vector.reshape(-1, 1),
                    subject_run_feedback_neural_rdm_vector,
                )
                fit_capture.fit(
                    subject_run_model_rdm_vector.reshape(-1, 1),
                    subject_run_capture_distance_vector,
                )

                # In this case, regress out the feedback history model from
                # neural and "Capture Distance" RDMs
                residual_neural_vector = (
                    subject_run_feedback_neural_rdm_vector
                    - fit_neural.predict(subject_run_model_rdm_vector.reshape(-1, 1))
                )
                residual_capture_vector = (
                    subject_run_capture_distance_vector
                    - fit_capture.predict(subject_run_model_rdm_vector.reshape(-1, 1))
                )

                subject_run_rsa_corr = compare_rho_a(
                    residual_neural_vector, residual_capture_vector
                )[0][0]
                subject_run_rsa_corr = np.arctanh(subject_run_rsa_corr)
                subject_rsa_dict[rdm_model].append(subject_run_rsa_corr)

        subject_rsa_result_dict = {
            "subgroup": subgroup,
            "subject_id": subject_id,
            "cluster": cluster_name,
        }

        for rdm_model in feedback_history_model_rdm_list:
            rsa_name = f"capture_distance__{rdm_model}"
            subject_run_rsa_corr_list = subject_rsa_dict[rdm_model]
            assert len(subject_run_rsa_corr_list) == len(run_id_list)
            subject_mean_rsa_corr = np.mean(subject_run_rsa_corr_list)

            subject_rsa_result_dict[rsa_name] = subject_mean_rsa_corr

        rsa_result_dict_list.append(subject_rsa_result_dict)

    second_level_RSA_df = pd.DataFrame.from_records(rsa_result_dict_list)

    return second_level_RSA_df


# A helper function to perform permutation tests on dataframes from above
# since the model RDM column names are distinct from previous second-level RSA results
def perm_test_on_capture_distance_partial_correlation(rsa_df: pd.DataFrame):
    feedback_history_model_list = [
        "recent_2_trial",
        "recent_3_trial",
    ]

    for rdm_model in feedback_history_model_list:
        rsa_name = f"capture_distance__{rdm_model}"
        print(f"Capture Distance <- {rdm_model} RDM (regressed out)")

        print(f"\tmean (z-scored) r = {round(np.mean(rsa_df[rsa_name]), 3)}")

        res = permute_1samp(rsa_df[rsa_name], alternative="greater")
        print(
            f"\tt = {round(res.statistic, 3)} (P = {round(res.pvalue, 4)}) {'*' if res.pvalue < 0.05 else ''}"
        )

        d = np.mean(rsa_df[rsa_name]) / np.std(rsa_df[rsa_name])
        print(f"\td = {round(d, 3)}")

        print()

In [18]:
# (1) MiOG cluster, Discovery group
partial_capture_distance_second_level_MiOG_disovery_df = (
    create_partial_correlation_capture_distance_second_level_rsa_df(
        feedback_history_cluster_mask_list[0], "discovery"
    )
)

partial_capture_distance_second_level_rsa_output_dir = (
    rsa_result_output_path / "partial_capture_distance_second_level"
)
os.makedirs(partial_capture_distance_second_level_rsa_output_dir, exist_ok=True)

partial_capture_distance_second_level_MiOG_disovery_df.to_csv(
    partial_capture_distance_second_level_rsa_output_dir
    / "discovery_MiOG_partial_second_level.csv",
    index=False,
)

print()
perm_test_on_capture_distance_partial_correlation(
    partial_capture_distance_second_level_MiOG_disovery_df
)

Processing 1_MiOG (Subgroup discovery)...

Capture Distance <- recent_2_trial RDM (regressed out)
	mean (z-scored) r = 0.267
	t = 7.67 (P = 0.0002) *
	d = 1.98

Capture Distance <- recent_3_trial RDM (regressed out)
	mean (z-scored) r = 0.257
	t = 7.886 (P = 0.0001) *
	d = 2.036



In [19]:
# (2) MiOG cluster, Validation group
partial_capture_distance_second_level_MiOG_validation_df = (
    create_partial_correlation_capture_distance_second_level_rsa_df(
        feedback_history_cluster_mask_list[0], "validation"
    )
)

partial_capture_distance_second_level_MiOG_validation_df.to_csv(
    partial_capture_distance_second_level_rsa_output_dir
    / "validation_MiOG_partial_second_level.csv",
    index=False,
)

print()
perm_test_on_capture_distance_partial_correlation(
    partial_capture_distance_second_level_MiOG_validation_df
)

Processing 1_MiOG (Subgroup validation)...

Capture Distance <- recent_2_trial RDM (regressed out)
	mean (z-scored) r = 0.285
	t = 6.502 (P = 0.0001) *
	d = 1.679

Capture Distance <- recent_3_trial RDM (regressed out)
	mean (z-scored) r = 0.269
	t = 5.941 (P = 0.0001) *
	d = 1.534



In [20]:
# (3) IFG cluster, Discovery group
partial_capture_distance_second_level_IFG_disovery_df = (
    create_partial_correlation_capture_distance_second_level_rsa_df(
        feedback_history_cluster_mask_list[1], "discovery"
    )
)

partial_capture_distance_second_level_IFG_disovery_df.to_csv(
    partial_capture_distance_second_level_rsa_output_dir
    / "discovery_IFG_partial_second_level.csv",
    index=False,
)

print()
perm_test_on_capture_distance_partial_correlation(
    partial_capture_distance_second_level_IFG_disovery_df
)

Processing 2_IFG (Subgroup discovery)...

Capture Distance <- recent_2_trial RDM (regressed out)
	mean (z-scored) r = 0.185
	t = 4.85 (P = 0.0004) *
	d = 1.252

Capture Distance <- recent_3_trial RDM (regressed out)
	mean (z-scored) r = 0.172
	t = 4.87 (P = 0.0003) *
	d = 1.258



In [21]:
# (4) IFG cluster, Validation group
partial_capture_distance_second_level_IFG_disovery_df = (
    create_partial_correlation_capture_distance_second_level_rsa_df(
        feedback_history_cluster_mask_list[1], "validation"
    )
)

partial_capture_distance_second_level_IFG_disovery_df.to_csv(
    partial_capture_distance_second_level_rsa_output_dir
    / "validation_IFG_partial_second_level.csv",
    index=False,
)

print()
perm_test_on_capture_distance_partial_correlation(
    partial_capture_distance_second_level_IFG_disovery_df
)

Processing 2_IFG (Subgroup validation)...

Capture Distance <- recent_2_trial RDM (regressed out)
	mean (z-scored) r = 0.227
	t = 5.689 (P = 0.0002) *
	d = 1.469

Capture Distance <- recent_3_trial RDM (regressed out)
	mean (z-scored) r = 0.229
	t = 5.456 (P = 0.0002) *
	d = 1.409

